In [ ]:
# notes
# machine learning works with tables
# rows are examples and columns are atributes
# tensor = n-dimensional array

In [ ]:
# in genDL repo
# hi.true = high expression, true sites from highly expressed genes
# lo.true = low expression
# fakes have the same GT/AG site, but everything else is random
# write something that takes the true set, fakes, and a fasta file as input
# fasta is the thousands of examples of true or fake sites
# use ex3 data for testing. ex5 has one replaced nucleotide.
# tttcAG, where tttc can be tttc or cttt in ex3. AG does not change.
# 3 files you want to use: hi.true, lo.true, ex3. trying to identify acceptor sites in ex3
# when labeling data, run only acc or don at a time.
# labels not for acc, don, they are positive/negative
# 1 for positive, 0 for negative

In [ ]:
# one-hot encoding
# A: [1,0,0,0]
# C: [0,1,0,0]
# G: [0,0,1,0]
# T: [0,0,0,1]
# https://d2l.ai/chapter_preliminaries/pandas.html#reading-the-dataset

In [23]:
import pandas as pd
import gzip
import torch

def encoder(file):

    fp = gzip.open(file)

    one_hot = {
        'A': [1,0,0,0],
        'C': [0,1,0,0],
        'G': [0,0,1,0],
        'T': [0,0,0,1]
    }

    encoded = []

    for line in fp:
        onehot_seq = []
        bstring = line.rstrip()
        line = bstring.decode()
        if line.startswith('>'): continue
        for n in line:
            onehot_seq.append(one_hot[n])
        encoded.append(onehot_seq)

    return encoded
    
encoded_true = encoder("data/acc.ex3.fa.gz")
encoded_false = encoder("data/acc.fake.txt.fa.gz")

inputs_true = pd.DataFrame(data = encoded_true)
inputs_false = pd.DataFrame(data = encoded_false)

label_true = [1]*len(encoded_true)
label_false = [0]*len(encoded_false)

inputs_true["label"] = label_true
inputs_false["label"] = label_false

encoded_data = [inputs_true, inputs_false]
encoded_data = pd.concat(encoded_data)
# need to convert onehot encoded lists into a tensor?
torch.from_numpy(encoded_data)

inputs, outputs = encoded_data.iloc[:, 0:42], encoded_data.iloc[:, 42]

print(inputs_true.shape, inputs_false.shape)
encoded_data.shape

X, y = torch.tensor(inputs.values), torch.tensor(outputs.values)

TypeError: expected np.ndarray (got DataFrame)

In [2]:
# mix in the reals and fakes for the input
# two different labels
# titanic who lives who dies ML practice